In [332]:
from tool.preprocess import Preprocess
from tool.s3_connect import S3_connector
from textrank import run_textrank
import pandas as pd
from wordToVector import WordToVector
from tqdm import tqdm
from database.db import *
from tqdm import tqdm
from scipy.spatial import distance
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
import numpy as np
from tqdm import tqdm

In [2]:
s3_connector = S3_connector()
run = run_textrank.RunTextRank()
keys = run.key_extractor()
w = WordToVector()

In [326]:
data = select_not_null('정치','2020-11-02')

In [327]:
vectors = []
key_sentences = []
keywords = []
indexs = []
for idx, key_sentence, keyword, date in data:
    if type(w.vectorize_word_list(keyword[1:-1].split(","))) is float:
        continue
    indexs.append(idx)
    key_sentences.append(key_sentence[1:-1].split(","))
    keywords.append(keyword[1:-1].split(","))
    w_vector = (w.vectorize_word_list(keyword[1:-1].split(","))).tolist()
    vectors.append(w_vector)


In [328]:
data

[('100-023-0003573085',
  '{"다른 민주당 관계자는 “당헌과 함께 수정할 부칙 조항에 이번 전 당원투표 등을 거쳐 결정한 서울·부산시장 보궐선거 공천 부분은 추가로 전 당원투표를 할 필요가 없다는 내용을 넣을 예정 이라고 했다","민주당의 한 의원은 “서울·부산시장 선거에 공천할 수밖에 없는 현실적 이유 때문에 의결 권한도 없는 전 당원투표를 명분쌓기 목적으로 활용한 것 이라며 “당헌을 무시했다는 비판은 피하기 어려워 보인다”고 했다.","더불어민주당의 내년 4월 서울·부산시장 보궐선거 공천 및 당헌 개정 여부를 묻는 전당원투표 안내문"}',
  '{투표,당원,민주당,당헌,공천,서울,부산,시장,선거}',
  datetime.date(2020, 11, 2)),
 ('100-082-0001039759',
  '{"리얼미터 조사 기준으로 이낙연 대표는 6개월 연속 하락하며 처음으로 단독 1위를 내줬고 이재명 지사는 공동 1위이기는 하지만 첫 선두에 올랐다","더불어민주당 이낙연 대표와 이재명 경기지사의 ‘빅2’ 구도가 ‘3강’ 구도로 재편되는 흐름이라는 평가도 나오다","각각 21.5%로 공동 선두를 차지한 이낙연 대표·이재명 지사와의 격차를 오차범위 밖이지만 단숨에 좁히며 3강 구도를 형성했다"}',
  '{대표,지사,이재명,이낙연,조사,의원,구도,리얼미터,총장}',
  datetime.date(2020, 11, 2)),
 ('100-421-0004964008',
  '{"2일 여론조사 전문업체 리얼미터가 지난달 26일부터 30일까지 전국 18세 이상 유권자 2536명을 대상으로 실시한 10월 4주차 주간집계 결과에 따르면 문 대통령의 국정 수행 지지율 긍정평가 은 전주 대비 0.7%p 포인트 내린 44.9% 부정평가는 1.3%p 오른 50.9%를 기록했다","긍정 부정평가 격차 오차범위 밖…부정평가 1주 만에 50% 복귀 리얼미터 제공 © 뉴스1 서울 뉴스1 문재인 대통령의 국정수행에 대한 부정평가가 1주 만에 다시 50%대를 기록한 것으로

In [329]:
df = pd.DataFrame({'news_id':indexs,'key_sentence':key_sentences,'keywords':keywords,'vectors':vectors})
df.head()

,news_id,key_sentence,keywords,vectors
0,100-023-0003573085,"[""다른 민주당 관계자는 “당헌과 함께 수정할 부칙 조항에 이번 전 당원투표 등을 ...","[투표, 당원, 민주당, 당헌, 공천, 서울, 부산, 시장, 선거]","[-0.15592455863952637, -0.014079248532652855, ..."
1,100-082-0001039759,"[""리얼미터 조사 기준으로 이낙연 대표는 6개월 연속 하락하며 처음으로 단독 1위를...","[대표, 지사, 이재명, 이낙연, 조사, 의원, 구도, 리얼미터, 총장]","[0.19289469718933105, 0.29414889216423035, 0.3..."
2,100-421-0004964008,"[""2일 여론조사 전문업체 리얼미터가 지난달 26일부터 30일까지 전국 18세 이상...","[평가, 부정, 전화, 수행, 국정, 기록, 조사]","[0.01197656150907278, -0.10067243129014969, 0...."
3,100-421-0004964731,"[""검찰 역사상 가장 수치스러운 사건…스스로 돌이키지 않으면 국회가 나설 것 김종민...","[검찰, 위원, 최고, 조사, 차관, 대통령, 국회, 수사]","[0.7033575773239136, -0.09183067083358765, 1.2..."
4,100-009-0004686516,"[""재산세를 완화할 중저가 1주택 기준으로 정부와 청와대는 공시가 6억원 이하를 민...","[재산세, 이하, 정부, 완화, 기준, 민주당, 청와대, 주택, 대주주]","[0.24903562664985657, 0.10665787756443024, 1.1..."


In [330]:
# ## cosine similarity ###
# cosine = {}
# for i in range(5,len(df)):
#     cosine[i] = 1 - distance.cosine(df['vectors'][4], df['vectors'][i])
# cosine_new = list(cosine.items())
# cosine_new.sort(key=lambda x:x[1], reverse=True)
# print(f"base : {df['key_sentence'][4]}")
# print("")
# for i,j in cosine_new:
#     print(f"{i} {df['key_sentence'][i]}  | {j}")
#     print('')

In [195]:
# cosine = []
# for i in range(len(df)):
#     temp = [i]
#     for j in range(i+1,len(df)):
#         if (1 - distance.cosine(df['vectors'][i], df['vectors'][j])) > 0.9:
#             temp.append(j)
#     cosine.append(temp)

In [382]:
from queue import deque

In [383]:
dic = {i:[i] for i in range(len(df))}
for i in tqdm(range(len(df))):
    for j in range(len(df)):
        if (1 - distance.cosine(df['vectors'][i], df['vectors'][j])) > 0.97 and i!=j :
            dic[i].append(j)

100%|██████████| 1162/1162 [02:47<00:00,  6.92it/s]


In [388]:
dic[649]

[649, 0, 252, 815, 858, 921, 1132]

In [389]:
import copy
test = copy.deepcopy(dic)

In [370]:
# temp = test.pop(1161)
temp_set = set()
for i in range(len(temp)):
    temp_set = temp_set.union(set(dic[temp[i]]))

In [371]:
temp_set

{21,
 84,
 89,
 94,
 150,
 154,
 191,
 202,
 211,
 246,
 248,
 250,
 275,
 289,
 333,
 403,
 417,
 528,
 529,
 536,
 542,
 550,
 578,
 583,
 588,
 673,
 745,
 787,
 800,
 823,
 824,
 862,
 873,
 880,
 888,
 892,
 925,
 929,
 944,
 984,
 1021,
 1028,
 1123,
 1161}

In [379]:
print(df['key_sentence'][1161])
print(df['key_sentence'][1123])
print(1 - distance.cosine(df['vectors'][1161], df['vectors'][1123]))

['"추미애 법무부 장관 왼쪽 과 윤석렬 검찰총장 사진 뉴스1 파이낸셜뉴스 추미애 법무부 장관의 수사지휘권 발동과 인사권·감찰권 행사를 반기를 든 일선 검사들에게 사표를 받으라는 청와대 국민청원이 게시 3일 만에 27만명 넘는 동의를 받았다"', '"2일 오전 7시 기준 국민청원 게시판에 올라온 커밍아웃 검사 사표 받으십시오 란 제목의 게시 글은 27만명 넘는 동의를 기록했다"', '"이 검사는 지난달 28일 검찰 내부망 이프로스에 추 장관의 수사지휘권 발동과 감찰권 행사를 비판하는 글을 게재했다"']
['"주호영 국민의힘 원내대표는 페이스북에 “현직 변호사가 옥중에 있는 금융 사기범의 희미한 기억과 주장만으로 현직 검사의 실명을 거론하고 추 장관은 자기 부하인 현직 검사들을 ‘심판’하고 있다”며 추 장관을 직접 비판했다"', '"국회사진기자단 추미애 법무부 장관에 대한 일선 검사들의 반발이 이어지자 더불어민주당 권리당원 게시판에는 “정치검사들의 발광” “내쫓아야 한다” 등 과격한 비판 목소리가 많이 나왔다"', '"게시판에는 주말 동안 “검찰 개혁을 하는데 정치검사들이 발광하는 것은 당연하다” “ 반발하는 검사들을 모두 한직으로 쫓아내야 정신을 차릴 것” 등 강경 대응을 요구하는 목소리가 다수 나왔다"']
0.9292690912543196


In [361]:
temp_set

{21,
 52,
 84,
 89,
 92,
 94,
 117,
 127,
 150,
 154,
 175,
 191,
 202,
 211,
 212,
 246,
 248,
 250,
 275,
 289,
 333,
 345,
 372,
 384,
 403,
 417,
 431,
 440,
 488,
 515,
 518,
 528,
 529,
 536,
 542,
 550,
 552,
 562,
 578,
 583,
 588,
 601,
 673,
 677,
 728,
 729,
 745,
 746,
 758,
 787,
 800,
 823,
 824,
 843,
 849,
 862,
 873,
 880,
 888,
 892,
 923,
 925,
 929,
 944,
 960,
 962,
 964,
 984,
 1021,
 1022,
 1028,
 1071,
 1095,
 1102,
 1123,
 1161}

In [351]:
# temp = [0]
# for i in temp: 
#     temp2=[]
#     for j in range(len(df)):
#         if (1 - distance.cosine(df['vectors'][i], df['vectors'][j])) > 0.9 and (j not in temp):
#             temp2.append(j)
#     if not temp2:
#         break
#     else:
#         for k in temp2:
#             temp.append(k)
# print(temp)
# cluster = []
# cluster.append(temp)
# new_set = new_set - set(temp)

In [309]:
temp

[0, 14, 1020, 1058, 1089, 1090, 1147, 23, 425, 917, 860]

In [312]:
1 - distance.cosine(df['vectors'][2], df['vectors'][5])

0.9666036223637304

In [293]:
df['key_sentence'][425]

['"2일 여론조사 전문기관 리얼미터에 따르면 더불어민주당 지지율은 지난주 대비 0.3%포인트 하락한 34.8% 국민의힘은 1.6%포인트 오른 28.9%로 집계됐다"',
 '"리얼미터는 당정 지지율이 동반 하락한 데 대해 주식 양도소득세 대주주 기준·1주택자 재산세 완화 등 세금 문제 전세 실종과 관련한 부동산 문제가 영향을 미친 것으로 보인다 고 분석했다"',
 '"여당 지지율이 떨어지며 문 대통령 지지율도 덩달아 동반 하락해 부정 평가가 다시 50%대로 진입했다"']

In [134]:
# values = [i[1] for i in cosine_new]
# plt.figure(figsize=(20,10))
# plt.plot([i for i in range(len(values))][0:200], values[0:200])

In [149]:
# #dbscan
# db = DBSCAN(eps=0.1, metric='cosine', algorithm='brute').fit(vectors)
# core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
# core_samples_mask[db.core_sample_indices_] = True
# labels = db.labels_
# from sklearn import metrics
# # Number of clusters in labels, ignoring noise if present.
# n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
# n_noise_ = list(labels).count(-1)
# n_clusters_

0